# Gleason Pattern Spotter
The Gleason Pattern Spotter is a web application that takes a prostate cancer image, divides the image into a grid of several boxes, and then detects the Gleason Pattern for each box. 

<font size=3 color=red>!!! Please be aware, that the detections are from perfect and should not be trusted !!!</font> 

This application is the result of my approach to the [PANDA Challange](https://www.kaggle.com/c/prostate-cancer-grade-assessment). Due to bugs in my code I wasn't able to submit the approach yet. This application is designed to inspect the image, the boxes and the detected patterns.


By pressing one of the buttons "Karolinska Biopsy" or "Radboud Biopsy", a random image of the one those Data Providers will be taken, processed and displayed below. (Additionally you can upload your own Biopsy by using the "Upload Biopsy" Button).

The image and its predictions, will apear below and can be interactively analyzed using the toolbar in the top right corner. 

| Icon | Name | Description |
| --- | --- | --- |
| ![](https://docs.bokeh.org/en/latest/_images/Pan.png) | Pan | The pan tool allows the user to pan the plot by left-dragging a mouse or dragging a finger across the plot region. |
| ![](https://docs.bokeh.org/en/latest/_images/BoxZoom.png) | BoxZoom| The box zoom tool allows the user to define a rectangular region to zoom the plot bounds to. This is done by left-dragging a mouse, or dragging a finger across the plot area. |
| ![](https://docs.bokeh.org/en/latest/_images/WheelZoom.png) | WheelZoom | The wheel zoom tool will zoom the plot in and out, centered on the current mouse location. It will respect any min and max values and ranges, preventing zooming in and out beyond these values. |
| ![](https://docs.bokeh.org/en/latest/_images/Save.png) | Save | The save tool pops up a modal dialog that allows the user to save a PNG image of the plot. |
| ![](https://docs.bokeh.org/en/latest/_images/Reset.png) | Reset | The reset tool will restore the plot ranges to their original values. |

In [4]:
import numpy as np
import pandas as pd 
import io 
import random
import itertools
import numpy as np
import cv2
import time
import openslide
import torchvision.transforms as T
import gc
from bokeh.plotting import ColumnDataSource, figure, output_notebook, show
from bokeh.models import Text, Rect
from fastai.vision import *
from PIL import Image as PILImage
from ipywidgets import Label, Button, FileUpload, Output, VBox, AppLayout, Layout

In [6]:
learn = load_learner('', 'trained_model_fastai_resnet50.pkl')

In [7]:
IMAGE_DIR = '/kaggle/input/prostate-cancer-grade-assessment/train_images/'
def getSampleImage(provider='radboud'):
    """Given a 'provider' and height 'h' this function will return an sample/random image
    from the training data."""
    query = train.data_provider==provider
    filename = train[query].image_id.sample().values[0] + '.tiff'
    info = str(train[query].sample().to_dict(orient='list'))
    image = openslide.OpenSlide(os.path.join(IMAGE_DIR, filename))
    
    too_big = True
    i = 0
    while too_big:
        w, h = image.level_dimensions[i]
        if w*h<2**26:
            too_big = False
        else:
            i = i + 1
            
    return image.read_region((0,0), i, image.level_dimensions[i]), info

In [8]:
provider_urls = {'radboud': ['https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..QobuH8L4elX4LhDS4doEaQ.BGghuL3l0Meap7QuA3gATGQqiYoG51BJHLcSiK9QvGGl8G8yPeMs43W_qDhll_H43MEpz22xumS-gweXhdkfJyfPIV-w8M_WOaFrrM22gE6h4Ka2U6nikAziOTGqa27oP3cVvS7Czl460IFVmbbhtOjQZ_euMtM82MuAgK-3wd6Cv29Jn5YmosvvDeqVyXqCOfNaTjKJvJaVjZhjDy5pPqvUfEnG0W_BwjcGGv4TGnSHToJ2vWQMmfef2KWWf733cLV2DcE6BV1NpDNNuSfE0FG7vWdSnkXmbzl_8VaWfW4xgLe18xX_BlqH9ywmoZWjPZnkTEDN5O7MdNYB4tPX_WBuj_lEI5YiUuyG-1PWDZV_1GLNOkOROi9rZu9Dur00ohkpONik4YfVWFrUG8tMSGK7bOpB5oVyfeqiG5jbLRko547hP8X6lmQlKpBkyNjWq2S8rrGw4m-m2bBqXKErzEW9dFHBAp7R7sOIZyznt8vCLO5Neh2D4XvCKK7eDsNwpyeLl42-GkikXt0qA_dG844PIgbYP5teEz-KHyNrY3ACGUsbRsUkaApZ3YtNi8QAsedwYOkgNMUibIepAr3BXDJuKhvbAfgyoF4puk7CsCycmwSdIi6dNiEDXFKn8F2XJYu4lLqtIpAnugy7aF41cQ.MNzhgJuM0wXtBySAcIrT1Q/345f3e86a3dcc26dce73ceeacc5a4a0a.png',
                                    'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..-9mvKGMkGOgXGUeN7IOWZg.r_FMQFwk3S4zZu_kqHC4DntHi0PO1fNa6A4-QhE-TLUnz6wrrbAnplUXa6YYFznYHl--HKFr5vdN1N7rJlKjDoWXEnkgKxa0eoceyqeL8sIKLbn6UZg8gsTAjQx-LCap1RKXj18ofgWkcrAK7Q8f4vJ97tW7rfpLpplWoaDojYzdy3J3jKLxZhruvcwED2Vk_QAuqSkdIL9Z2fL7ILZWyd1zxl0qTQzVLv_BmZ9OQDqqaZReHvjdx_ssNJui8UdhxUZPuAfhW7VP2uwhyIW8hwxON7nXBzjeMK43j9CjkEMPxLwnVOdno8IsoOakcXSe06dqVDlRA6B4swG9_QGhMMT-_nk3H7LwdP5mfth5SN-m3Bcaq_qbCNjJ3c_IKejufM8Zkb28Fu1rxlA8ftCbWngHqP2uE01mnCIstiepCY0j7-ls8xQXUxi-J62BXQIuM_TECWaTqYobZ3i2xKutytn_PMRe4Nkmw7bfSFANUAGLwsoJ8t59RclY5R8HAwosRLkfe2IlPjqArFnXH5EkbGyJE2XmzjtyEqAwkKdvSy5qj2Xq4e8tZCyRBjqaD3EDrSwrmz7LZDWKmSy8JBZmWCpiXdI_OdKiite3efwCV-0Sa0TlHG8BK4qGKEtClvUtIWx3WA69POAOMguJiEFcTA.fJlN2G5vfu3eC-sp-cNwhA/35f629e359399d425289242b698b3f66.png',
                                    'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..-Y29SKPNvrnxgQMALPb7Uw.Dgyv5hFe3X1-5A8RyPxRYjYALaqG4kTQpugp2W4LBBj67rNQWSBcnTJl7HjhdI64tR7aIRdxK13qFca7No2hdz2REtZOvVQrsxvHIgvZ353IU5tx_9urnlJob8mDp2nji4HwvuO8N_0ZtHV9AT7jOtJ5gjhb1bF7i3SIvsDK4jGfhI1QVyfqE5XkHHItuZ3_D2lGGfMyy_fL_mFk53YSmWHQ1DaMO4YK8HG74t9qgm400hcXLbi3YdyQg2pbcwtszRarItvfd8cCYSw2zWNN1aSYaehJSpQQb1XEWMxRENyFHBPIhe1SgXoHHKPirKWJgfaijVPr5FlV2u-lgW1kxZcYLPCTHpKwnGQeazTnnw2AsKdYKjrJj6MZ9hQdRWu9O4rYd_6-fOB4_D32XyV7LrOj05JC3jbv3o-ZPWdjuXVoI3Jj-CyDVeE1V9CG3YR6RBPUuBsAmvOh3sMOXEXoFTAfWCKvh0hT2sCv4rVaTNB0sQ1KbC1MWhH1cUSk3CymN8hoygpRiLqRXUhZY8n4R0-gt7Z4OsmRsWId1kSF4i3JjZMyedIP6YW5d3PE8RG1AUJCupx47wM3YePnA1MlEgO61jXRJp6mX0sUCAiOy5O-N8CGeKk0rD47AxEEU-U8CwPwhvTQrLh_oaLB1Vqpkw.UNHefEqlCjCBVyJKFZuvaw/36eeee47a868b8ac66b6ece9e6372984.png',
                                    'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..eGbso37GqduYG6VsIV7E4Q.1Zs_C8RLaO68PoQX9dUp2IzYxUBGaNfCSE6kg5ceVGnAYBc-VI5ujE5aHfR3y2U-eIwHJ5P7x1gmfxHFqertz8PSAPSnVHzFEvhqDALOQofTehrrgD7edCk0QbwEAQbOdTTg6NVlnd42rgRHVLiWWDvrG1NT3mqfL5eEnRX6DryGBgq4gA8nw9u9o9QDXwotEkXZAo3Qdnv6srd3ZTyhUtkk4GpBibRn6ZGpdzx5lGhQ3KZm_-K3v7GxbAXmUpQTio8WdAKUaK_tiujrb3Kt2zb756_H0OobynZPnj2y-RptMauuQ-N57GEXszab06YJps7ahAiv9HQ404sI3NLhdIE1uH_3URhJ7CA6nHmMxxCEnd1s_iGfQ8_N7NzSNJRxnVuhxDKEK84xF4_Wc4mShy5Vp7oD0S4saq0xad-l34JJYxLKaoBQ5me-oXuXfKQrCIzXB7JuR6Y3e3zSE1OV9siHHgQNTJSR3R2nHdX48s8pzpuw0gvOi_hFvtcArJRQur-8z5lInmtaZYNXqDa-8ACPAUEKglnM4rivEBd7mUNK1KUTwrQ6t6V0-AQv5jJp5vhuvEswg_mdhIUR8acaB-4r1mZ3ycOo7WwubatD1VyDDwvCAS__4tyLRp0uvpmM3jH0DJBtp-3gLtrsdjiKZw.Cp6W5BeGrS0hRaJligb50Q/58d13c5b7c1d86814e7a9451fef0d9d5.png',
                                    'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..Ae68e6LiUXGNRR6WUwWo5Q.fv25qyJGT6oAFkHMCjGDjagfQv4NTx23OlhK6DNUhDTXADtyoPUkgoVaIciqr4dG-EZfkWmk7MuFQoAOcDZ1CDHKWSgiqOfmYSI_0ygiG6Jx6jD7zndXCPlF6Oz_V4TZDCskPUJgIlXlcdkwupTZuPO0iH-OOFq4CU_nqYGiXaPJJUcX6uK1cCXlyJLpTPqqZwcydDWOBI3Lp7ya7H_Ifv2ftbF-DkBZhAD1v-ZTw0LGaUVlsOSBJfORswftPADvDL07YAecDhdPkPpxBH0IytC9bvIdrp2MJpckRMpt1TwuHCWykp3rRkuX5ior0U5OcfvBwsFH0zSeFxV9-yNsOtqFA27JNhTGpzPZznWKemKVcgdirCCCiliEsejA4IHBBl-gSDLlHOOCr5nUHdFA4kD1X_KbAo9h-_ozGal_193cY2Hdqz_uxL3W6VKQV7KLCkIukPtWSORwio5SCcICY5u-1Mk3UW3esoJFR6A8ja8TBma3_SyuZ-0U4_XT2sdl3hyyOhvVVkD704llu30DxoIpCXD6dQHT3P7JVqD4_vEve4JrphnR9ZZ86ROV8sCybIH2yAEGXYBT3JDz533uSH45e_Zti4NxxpiM61wFTRpaUSImAQirCQ_GAlKAdCIrEUncxQ6elzhRlXjtTSLETw.6gx8m2mvAAzgtKWVWikHfw/658be675223ffa5509203009eb7bba89.png',
                                    'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..-t1pQtsTAZ1ND_zSIKcBxg.sqVxn37moBbeYNowlsAnQrdGGP-atpnAl7lxPMyrkJr85LT9qutjUYhc58gHOYh5JRfwBsJrG3f-HD49uinjpOU28hkhqV98CiKfmkVyCVh_G7DP-vbUu91cwQTsCljqs6Sn-qH0j_eRF5lxsHhcMZOZ-SZRUQNOGPydFLfBHiOeg8b-HQ8vnwj9KhnWTLSHRg_j-z5suAPj_rkfQyu9psgwjilmvvNE1pHF8SUu-qRGDPP6oFfU0SoRCb0vXjAvWqCYdhCwHuAnn0miY6D7XEFXqjoMvIPVj_YmbT6Q-yPMFkcRMSFX25FHV1W8Hw9GysaFsXlrLp0TD0JPtYmE67cOrZpkRokJ49vDUr2bAQfzqDShrdrxShVr11ZmzI3LrynPjvSqzL78E90QZDyD1QLCLCJPO_6mHqq5u4aEmBqaPlzIF310rUS672mLXgutRK9rjSPnCN6XXhwJok-rq5aM5ZqYE-8uVGeNi8veF2q4nXRrZnT4173VCBqXTkye6pEGDszOg86HppNkfcDQYmSfeMr-HokuoI-HRXP0PIg4_F5U5tdjOmhb59Pe4jGX2_lnx6xDxlIq9yjcVjtO1l0OXMFLT7fPEop86oHVkNIzSTBFCOGY5JbYStnPOjgS_y7MUEMYw6fAOkXi4ZMg2Q.5HOPMnncPCYtEobe6Oel4Q/6a5cc0b66b8d4af9c13f1303c6de5ffb.png',
                                    'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..LKokldx2q-Moe5ARgL_PwA.IuVIndXw0DTRNs225M1mIGi6DpAEiLlpjAEAAWijx56MPzNL-GjN3SIjPI32tFQ3xcqrNPbWOsZdhlhgzSK6DxxwQLeO3ELKYGeMkEhjkvWBKeUwdvRUyZ8sRjqjV7r9AFEzV55uCVpFKj0wam7zBb0sYxL8IqtuhGfPXxpgBg9MMbLq_JH0_r23Qbq1OEX89gEZeXJ0cpicLk4QHKVjiFWyGg8yNaEpHovulwXtycf019mmmWXUnNCw1A_VuQN3dNNP7OSBWMQ5FTXY0e1S-GFj1e1NdlXzc2UEyZXRRrtGFPoACviro-YvdcJuzmzgYSf6gTaee8b8XDEiaoCQxPt4hOsp4BMczJQhPxPaPVAGjlDAaZPKXopI6RR1wMbm59Gs4RAugEEq-qFLPP8Cl7_41aQNp5ASLVeaBk5QFh8ksyjjRekumPhnXfqsVwxEekJFzyNNzpI_ubjldSsgiiXRSpK_oW5LxCwNcb1_Qqr0o3Kf0OG3B1zpVBgMol04wXhjfu4kVRAjOhcVhcL1qc1AmZ4psP59RcbmoMsCpxFU9tEmXmehwfVsSLqk--G80nw1qqy66a768CwXJOrhe6qNPTokcnDUbus87LgX3qkCighQj1Ns1hs86y5_8j67ZPVZFPJmvuq5Jr5MB36rew.rDB8kc9OGtIcmBkAxZFD2A/8a21fc0aed2fd06a8b3fe4824c53413e.png',
                                    'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..vv7mU0NTQdO-kHcW6lmLSA.6SMgjVRAd0bu16FdV1Asp2Pdec0wn5srYlwLRzq_itmLBCin-Dhnq4B6PRVwWR7eD5WMIiRTBeD9MaJGhvE1zg_yS-Kg6DcGh8eLN1yWhgiqIkVIHFMmgdhThRSxQMh9QR_wF0V9FMsd1r7iWeVVXPxC-zTo0axURyBXaV8ZTLAZ2eqOSv9F95RpdZEolTYho64ppzskYcBoMUvfU630hnsY-jVrM6tizYDKbdu7LlCpKOEhVFeM7ewvdqxmkyq13reIwjzAiUJ-Ut-8wyV67YQAmyzi0wzctEQajP5VXOgP1s-BbLHfhZazJc16oePcIrK1iBp1wV8kRw666RAy5okjuqtlkWdNvj2k9BmIijDWzFBIEev5YuH_u0F0py3YiA3z6xmLw8Q-i8MvxEBpJmGJKRSdbAQ_9BTvUcE8IWVXeX24Hu6eFQPEGo686-wrH0lc-_L2oinaTM0DaRZ3-yz1n7J5NLMKZ8UnV7Maq2X-_JQxfi0ZfY3Iw7BfkAJ_5T0dc18octgN2qLzdO-OrmkmlRlrw3duo9DbzQSehyHJincbTB80UAbPjnbUozda_EYqKixzRcf6jqv9Apr8lxlYSMLTO6fYyOL7LIV5incm2cLdEj72RVZI3vVAC6IuE24i8ERgn2YsdRUp7fjDTQ.ingfSPx7678UGZyt8J_siw/b0ee38976e71dfbfba705974713ee612.png',
                                    'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..SH9GsJaGmBOJZyBVyLqftA.BrV-lslXUDhU_cQZTGk6MGfZajsupaplkG7n7zxXOtkoNHXgH5YELzsbKhAHcPrOdk7a2et7QUZKyT1t9yvvrg6gXdmuArh8mQr9POhlN6mgIyhlKFcmxLkIsN8nDbdmV4Eo9GrxMZ5MMxZ-eZneblKyfPc5kMznqBVeFiAmB4-ZQFzMsR1fIqOW8FB6-ZU1nOpVou7vL__O4OaXSXRZnlSFgxKM0t6u25uElvH9kYAmajOvJHPqDO-dozSZRnDwxa2ExN-SVMUvKqR-_OAiuZAk7a_7gStZsZ4kg0r5ZsMVO2oCZ_PO3A6sgJiwNLMrQ3KYCu1bbDQbA1N1EUKmeGjb5gj70MojgRiW1ejW3RgWZf1j3G2_LazSGbRRLKqloB6YEJrBOot1ZA7GE2G0M-7Vz1nnA1Y-iLbirPfpUj4xvsiG5I-WdSlmh09G2PFN5P8l51j4JslU-VT3R_2lknlsubjaVrXqrXVxXnkOiuHpAKncex9I_6ErWgz99C5PWk-Dt54ahtBPX2Nug0E0VKzhQOAge0nzvu9APG6MhhUVRTdRnAWuNpQAamP-9db00INEn5JlOaF0759jRWcuGneEr0x-v-uQrZ6fzBSbyTjJnJpT9J_nH41h2oKg3ejAo5y_0coZ9Loy5_PH7iqq1w.45lPU9UwLulx7gX77xMoQQ/c306e9bf52ed33801003d17ae3f60d32.png'],
                 'karolinska': ['https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pF9GtmTFtUA4ainHZKBpEw.TSI9YEu_nPYOU2d2Uxis464J3Sf88h71Mf30BDpc0GI_YDclEQQI7UOJZxOg8qNhCK2MURM88ias1R5W0-tE19-XF-tCMUn1q0WcY-jUnlYsfWPmZCdT_kN2GD0a17pEFOd7F42ZTlajh_WafalykzBxjp9swPyi0ZUJ60bD_Pg2TJ-cBAV-gSEGQTMMTD5PX_65xOKu_ECvanZ6fEjHWJEplxe8dWQaUK6tPZsxThVTpCOPcC7TBG6UX0gNq9y0sjft0jPSJYbsyXYO1uQoWYlYGktX1NyiFtYI8QvIZcehWbOm0xQ54mf7K788ls1HsF2lH3Qyo9ZRQhnjZsV1_5hfKwTEGn0BaimAKtYH5TNoyIqdG4PGEfzBdcRbaTx8Ihf4-EiaQFL-6a8DcrVR7xkhYlKLAU3ariwEjUX2KPv1BQwV83fdN-DMrWa_GiQCKbE0q7W-vuz6FJwDvxqaKQAy1nnO69BdmlsLKPPUfPY76yRc52wjZHe5O16hv22F9Egv2_JVNl3lRYmMx1Uk8mqAYJZBHoy5x5mjnluU4Rhy3Y3IuWGJVkQPiAOUrtvKCA7O_3MO1JiRgsvbnWdO6qS_y9t35VH0hMDxMmb93_QAtTbn9CC9WhIdvtSW5VQCIu5tLHsc8sfRZHQ96p5WRw.DyEkwwBwVLgrkbSl3NvyZQ/0d7666fdc3372c68a080c612081e2b45.png',
                                'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..JI1FHEJfdPZO5R5y50U-DA.OR0c3Bp-efzGkEkfdAiOvVV0gcjku_TZxEhvXjM1OLFj8GW6jjyU7h2uogrCKQ7iECqAZ9OAxq_bOJkdFU0LmBNXFK9g_f7w5Uab8_WfS7LDUMm15DiPmebja3hKEq5IG6rdsj6BtqS4xTR9eFkILPO7cJDTV4QFFybbR3rddRUxKUHS_SISdOAi4BiTWzvx2MUhI7LNQSWIhNlnJcbIfVCWzV2_RM7dwj7u3h0lZ9s4yiVckKFJ9G9_6IYTtHP0ai68AlqUGZHfaHQy0QGm9J7tpEoSydFXNOQLvkh9StNZb1InF2ATmIb8VXkfDzCjyceMIx8M21i7RqqXVpLytVyGtS5olyvTbvuvvH6j1CBNI6j33rBUnFsfFArRtUgozNQimqxhtsaRH_w89L5e3hwzHPb7TyDFQhD8BsD7yvp07YIInKz0W3CWUwMuV397Mvueq_XPqQgnU2IEc7UMMXRq3gR7inZAgEMmvxzKijaJIrToFWOazCux7DOzhSA2ECA0HSH02KOtWKtgij2tvHDScIPgIf4Sf7dAp-f4HRurz1JdYYS9hN2KqLWl6ZH1HbBYutRrLkbwvDhXQom_0VPsMH8XLn26JoWlTLhtPseSiCLKPWnjmkEuiG32TDm8-AmuUhWOpI-RVs10prJpRA.RPusjJwkPss3xA-W0onjaA/2665dd5db932525f171f931c16112c65.png',
                                'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..s4luTK_KL9rk1wmLdIGRcg.nt29OBQILnh649T4OUaMjJZ1PKzQ2r-_T-QovTXvUDAxLWGjdVLPin70aLxE-1PPvvfFfWAeFNsGGkYCVmuh3wxalwDAQouCHn1LBFBlQmTRkS4gnflpBgVkrVwGOU5H2JBjKyf5U-IWnuCp6wIlQmx3LiMkIjK0KssiW9DrA2-TLdqOIW8kirYVWSR0YGafa73Mny8mYbllMGreMiOOz9o2wxHZrbJH2ZOw0kJo2I5Lt7278o6QyaHlFj3-IxHv9shIURR-s_mSjr-YP7OpK-x1fCEzFGAE-7ybyzqySagCuBi3IhDm2PxGaH2stHaFgCToHD7W9scD45bVAmNhBet1K1YBh7VZXCrjJskiKkzVl5KgU-JKm9SYB35AGupk9ZQNgCeCzlE9Q--YwHmzt3-Qq44pg-pT7ttO9PhMAiWXHt1KEvXSMRBSqsuqHR5_VB6MfphcdgaaPeF92-kWXDCTy_3N8a20GPCp-X9lBTwHajBwd09cfCvVhiYsj_dIicgTiTsDIe0rDBD2j3jXhlnzW7-yn-1iWOL1sXggqi58Wkc69YHv4_wc5aRnCRRFjkVKa81HRX9HluO6UloYSXoRxfrjdHhH3rOwC4ZoQB323EALMRbsx_U__uUvdmEOzX_TdGTyAxLK755oJFKyHQ.Gy52qS3sYzLzEAuuC8hp9Q/5d35e9a318e04684020a42c3a2ca19fa.png',
                                'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..3FsB1cy3ka9Dj2pfCTNGPQ.tnOr-zLl1WXqxafFm6RtZ-IP-9r1CdiztzrO1UAdhcPikMbU3RV8uZ344IumnyXiekRm8C2sEK5hH2wnd3uzkUYpxzoQsl91wUgO20M9FxdiQREcXjkDAj5L4wE_hBTSC0JbtHXO0kfxyp8--Uu_z3PrrRPYTPaFCGMtUiutlAN9KnZq_GvcsmKx5yuVWnBOnjbmfkqOMDoCPt5lOMb6gcmUWjNWct6WYylZTynOQ9z6JXR6CLzs0jP-q52vqhoUdfM-j_aJ2s3v5QxDtW_Eqy5-l6MlhhZzi-nxgdlq8sMqicDoy2Va2pn1-s_1ijkgEFEOp1aA-eYYFy5nQJBhXTY-_tT7CTVoQNVKlQfiIxN7UivMpYCL5ZigpqGGLl4B-ElVAirOCRC1FHn_LvliQioAduKv8WFeqkYtgWHnB18DridMXQPCo96_6xq98F4vuV6ASpe7vB3EoKK79KH4AV5Se-jWSrsIW5AZ01DAWvS39ZBhyBhjx6ZCYoKCDWMyHNjF4iwD17SJ2LeH0HcKkF7v7J836kzjsT1UPbD4bxvsyMH7ref6d7HcCo1CPXWt46GzaIw2mz5FaLa48Gfxres27fBZJOMZNKoH6i3bDqc7eTdI_je5wAel2muxB3j01Jy82a-feq6Sz6Ewvg4_Nw.7_DINTu25VcwMO_RhG-_qA/89d137c7fb29df7596e6824181b7d5c9.png',
                                'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..2W3pv4A0MYW7QBRvIzlrdg.oKjE-GnVArlUu7UCDSPtL5CQOWzoCRhRGXNmC08JgrgfHwJ6Q65sFc84kxBOdpPxh7uphU-ZnKx8eMPsk3Aa0jpO6YFYIMMF9wpcMvAlIsolIrZdkZu-lGUEJ2ctOhEaag6aEPwYvvYRaDh4VD40cPQbxjF6DR8RSkncHJu_9NphJxNmR3yjTuNr9XUGioNjDNmt-v_bCEIcymdQhcSO5wJG-vp8l3uk3Olv2X7ko6uXeBIGunUFTJOZC-h-OtY9JxWSJx4W5u_iRF2QMoC5MCWwtJOlyIfGyFNvWu4IRs3o19drufoVuvkc_51m1nc-XSCwtKTfptw65QIkd2-rn7piiOVe8cm3KMRW_ol4nza_VUZYFvS_bNVGF8_sEDGy0W9hqVR6kfCPbUxzuUIvaIkeM90ETC81ehmHv0hocYfffcglr4cHdBjctqZ5hvMN2s3_DZlAZ-ri5si0IkEO3F1nu_vp-MzWfnsjgdub4gktkhz7AiMJts4cgGhIOYA9cWIKNKMULsYa1qR2FLqeHBBc1WiX2t8GKod3wBewhxeU0-LeNijZ-HfGznu9JB414alTO4FX70u9jKymHh-B28ByK5Jz_tRiu8kY9kGraJfxDMnGbWu05BJlWvOtPIdmw8IAneWwoNKPTqjjJEQKrw.XTTUUkXHgWQ5yTv6U9JtOQ/a6a07e971c280f5cee808a92a5b06e70.png',
                                'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..rtzj-CdkOhHjM2xP3dfe9w.sWHCuEA8-GmsrZKL3ZDDRQ9_SjVyCEGN9jyg7rb5j03WOeLX0jMI_kLhRWSktQZno2gXjsoDoaBrkv6YaDoBTBHZVhjwrICy-nS4D5gFL_2QZlAlZLdiOX3FY5gdnH2tUSaKJWURoXMPM3cmOHKZnCuoBrm08bMycnEYPtcc21LV7OPDx_IQzceCv4_KB-uQcO-4qbd90qgUcIBxsYwJxbM7opdVV-cXC1eZs_OVHNU0yRRNX-dKyVgTzkFz7TBkuP6hgt_GBwwcIX58WBNCBvg9yxDAJqZcaYQeVFfuWTK9sXzmZFHPOrhMgBE7WDG8t2utsbRxLMI646jkBM_yvX2jtJGltRpK35CZe44IUZ2PhXr0IO73qFc30elC2rSyf5787BuFZ5Ytgxj5e7inOeD3gxzpJEgfJTaZaBS4QNKMO8CGaFWVddsUzgzis8OxXeuGpG5UyYOr7cIPLosfVlpusteZ2d7svXuvKgg_KU7Oc5X4aN40l3-U5ce1VDyFlqS1Ai9PH65FqBnINtLrwJCY6vjKQnsOTToCgeivmiabffwn-bExoBHsCceZsAsz46g_xH4BggRWnn3AINcLViQ1dIwnxrMgwbCgQvJB2EEMZ3p0mgvfEVNlJaui3Esq3BCLZ0pck8bx8hjniBor-A.ppgEp_FlJs2k4CVhm52gEg/acc79fc48cdb1a2c730a0b2c15b0355d.png',
                                'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..GonO7VTwL_0ve8pwgIsf8A.BKvr1fhB0uaSS7EhVMp6mCc0KwdhOhYNxS0BKgaadfaVL-3hzH1pWElpuwUyXNYkERNdyodg1iGUmiicTjEUsCbxKwQQfHIvyHMvfzAFk8xGVAdUfUMKkml9wh9TYLcS5_D3EHGoue2oO2UdSkvM2laA8OB0irJ9LH_kGtjrYB6HpjzgNbaubAyMyCvv_lfoFZUtUSR6ZqUqP4GIafDaqUzdT-Z2HJWC-BO2eCskpkQ5Bxo-7kjkSRK5ax0mybq5dbfaXcG78uPJLjak-m4_0ZJ6QmPVPv8209fQ5jK5ZYXXEBJApwUJIymHC41rKLRvZFC5J0UrR8AITzfy9gcnSqm2LrHriydnopZbCVnkr8m8k9BdukYxtdPUB4lXz8kBYpcHp1u2nltt1oS3Bt9sjA1_0DS6xsthTfd6VOHdmTQJR7fzLHwIlLbwqz8J66SfUxR5VZEVIkzJiDxQoK93oe4brYbmlkgktlMn9Y64NxB0gfPhGpkIZ3i88yPiAXyrZ3Ecro_dmQkTfsqrzeuo5U3zGl198Xhc8eHPpLK4UypnLpBXadXL3hF03OhYkM1tzzC7n8G42ASDwAoQybLZprzsV3To_ZIdVi1TlWyt9ZhcvwTeZrWf8zK5Po2l46lmhC1b6UOlfSqNZ5Dvg_BnnQ.wTnl4Ad9xOC1QIF0mDHmMQ/c4b7ca39cc464a7af7f80fb9f4bc05c3.png',
                                'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..e3KOrFyDImVxY0BzjXYXfw.RTn1nSLhygg8fxqAKyCCj1lXUiBVEk3x6UIP7n6NnGJ7JzeF6TByjp7-39zYVmEYpDvkgL5LbGpedHv7eQCBGQrnuxbL0KMpDM2P28bTq9GcZy2ffFi3UcdP9JonIJVluUiCCUlIYwXDgJgJmd7k5IuAF-t0XHGoO5zliPvUsGhgmq_XPN-ekt7lSDaukkhemYpATb7dcBwbtOYAwzfIpnzEEMhrgFMVMmq8HXZBLGLe0GKR1f0QZD9ZZ5IlWeW64xYbesgloZT60KHQWxYFCW1aov4T8M015PLcFny07fyg6sZ8VQP2Eqcc8HKcXU4xs8ZQw3vKJZF28rNn2ZFeUujH57pjwM14f1NuJ65NcjMHLuonPDiHHKgilx0T5q0J11dd7p9zLOHeVzcTKQZp1spP0n2mf_aHPGZwaK4YfT2RKDZ7KKORmgNDSGEJfPbFMaAMBC5rVQrE9m42keVeRVh44STDgCHwEqvMKa0uTeML5CNHBamYK0H6jxJ7SUPvHwhbuB696i7a6ZuwveJsBtIYmDskkQ1vNrOgWuhlnJQXcgerIwnHQgWmMtJk3rHScoNNckACARawaYLjmU57zTJ2hptmf3OAlxBc4gqjEDrZ8e95taUqyRagakr2RnWk73kO8DPooJ-WgDK1wsQhMw.Q8kE_4NUx4qt9wn4-SgMWg/fd1deb6f87696b6372ea9c9b109b4ccf.png',
                                'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..NJwqioWKjwkfhDf8__0wcQ.EGNiFwa4j-FwkLS_Sm1TahQ7LpA3WjRvkkUN2Gylkkp733mqtExeykMje4C4qzZwVafp8xT_SrMvFR3m97FKoq3GdNQ-Zwimo5bMudyz49q-DbATC_w49RW4Nxr7G3B1cUogE7SdmlXoMabeZY8cZURByIjREg7HMYpKN4yZ7fZy6SUQV1gXpYIpk5fcDxQr2XwGcOBS9K-nP0b4CP24NFgJkltHZr6R6mZciixIvaxIUFfYc9LxTGIb5IPVd8adw4J2PS2V2m8eyLZfs7TvsnvptzCn2tzVPOwo_B_zvlQEzjcUEWOSouHsqewZy1T0DL8e0GAeXKIBXo2PNq0wEh-M66nJANk2TEkP9YkI_-FJXpW-gaWXHvcI63-Q_qIvdWLmC84p_9fXh0gmOFgIUNEO304lE2ryOLgDMrjH1p52I5aUQxOL1Avuvj9P1FKs55IJAFJJAXdGkTCYjgR7l3BCQ2FaHkxEj4dRw7v19x1HBa15kscef--_T12BA96I3O0Gkhy1AXJLLnnIKHVepnRAFiT8FR9ap4wopk5r72zRJ8hfRBBND3Jc9276asjtdHGB39rr3cvkBp_ELl4LwgtpeKZ-sbUvqdz3PWgbXIGPyXLjoj-osaES5sSqmDVggJaze0AgNOzCWp7yJnqjNQ.NFtm9CVquopSJkX9UNnINA/fe1d0b156ddba991585791769dc9ede9.png']
                }

In [9]:
def getSampleImageFromUrl(provider='radboud'):
    """Given a 'provider' this function will return an sample/random image
    from the notebook https://www.kaggle.com/jackbyte/sample-gleason-biopsy-pictures."""
    url = random.sample(list(provider_urls[provider]),1)[0]
            
    return PILImage.open(requests.get(url, stream=True).raw)

In [10]:
def calculateGridCoords(w, h, 
                        max_box_amount=100, 
                        init_pixels=256,
                        max_overlap=0.5):
    """Given widht 'w' and height 'h' this function will return a list
    of the x,y coords of (square) boxes and the pixel size of those boxes.
    
    The size of the boxes will be 'init_pixels' if the amount of the boxes is
    max_box_amount. If not, the size will be doubled until the amount is less
    than max_box_amount.
    x and ys of thiswithin the
    limits of 'w' and 'h'"""
    pixels = init_pixels
    coords = [i for i in range(max_box_amount)]
    too_many_boxes = True
    while too_many_boxes:
        
        # find x and y positions of box
        xes = [i*pixels for i in range(w//pixels)] 
        ys = [i*pixels for i in range(h//pixels)]
        
        # Add additional boxes if there is space left
        # and the additional box doesn't overlap more
        # than 'max_overlap'
        if (w%pixels!=0)&((w%pixels)/pixels>max_overlap):
            xes = xes + [w-pixels]
        if (h%pixels!=0)&((h%pixels)/pixels>max_overlap):
            ys = ys + [h-pixels]
        
        # list of all coordinates that have at least pixel_score pixels that are on the mask
        coords = [(x,y) for (x,y) in list(itertools.product(xes, ys)) ]
        
        
        if len(coords)>max_box_amount:
            pixels = pixels*2
        else:
            too_many_boxes = False
    
    return pixels, coords

In [11]:
def maskoff(img, coords, box_size, thr1=200, thr2=200, 
            krnlr=3,krnlc=3, target_dim=(8192, 26368), 
            pixel_score=0.25, verbose=False):
    """Given 'img', 'coords', and 'box_size' this function will  to return 
    - a resized version of the image
    - a matching image mask for the resized image
    - a image mask for the original image
    - a list of the x,y coords of (square 'box_size(d)') boxes 
      that overlap with the foreground of the original sized mask"""
    
    resized = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
    
    edges = cv2.Canny(resized, thr1,thr2)
    kernel = np.ones((krnlr,krnlc), np.uint8)
    closing = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=3)
    erosion = cv2.morphologyEx(closing, cv2.MORPH_ERODE, kernel, iterations=1)

    # When using Grabcut the mask image should be:
    #    0 - sure background
    #    1 - sure foreground
    #    2 - unknown

    mask = np.zeros(resized.shape[:2], np.uint8)
    mask[:] = 2
    mask[erosion == 255] = 1

    bgdmodel = np.zeros((1, 65), np.float64)
    fgdmodel = np.zeros((1, 65), np.float64)

    out_mask = mask.copy()
    start = time.time()
    if verbose: print('[maskoff] resized.shape', resized.shape)
    out_mask, _, _ = cv2.grabCut(resized,out_mask,None,bgdmodel,fgdmodel,1,cv2.GC_INIT_WITH_MASK)
    end = time.time()
    if verbose: print('[maskoff] grabcut took', end-start)
    out_mask = np.where((out_mask==2)|(out_mask==0),0,1).astype('uint8')
    
    out_mask_small = out_mask.copy()
    
    out_mask = cv2.resize(out_mask, target_dim, interpolation = cv2.INTER_CUBIC)
    
    # list of all coordinates that have at least pixel_score pixels that are on the mask
    out_coords = [(x,y) for (x,y) in coords if out_mask[y:y+box_size,x:x+box_size].sum()/(box_size*box_size)>pixel_score]
    
    return resized, out_mask_small, out_mask, out_coords

In [12]:


output_notebook()

TOOLTIPS = [("Classification", "@pred"),
            ("Gleason_3 confidence", "@Gleason_3"),
            ("Gleason_4 confidence", "@Gleason_4"),
            ("Gleason_5 confidence", "@Gleason_5"),
            ("benign_epithelium confidence", "@benign_epithelium")]

COLORMAP = { 'Gleason_3': 'yellow',
             'Gleason_4': 'salmon',
             'Gleason_5': 'red',
             'benign_epithelium': 'forestgreen'}

def bokehplotImageWithPatterns(image, confidence=0.7,  filename=None, verbose=False):
    """Given 'imgage' this function will return a bokeh figure containing
    - the input image
    - rectangle boxes that have been classified a Gleason pattern"""
    
    # resize the image 
    while sum(image.size)>10000:
        image.thumbnail(size=[v//2 for v in image.size])
    start = time.time()

    if verbose: print(image.size)

    # transform to cv image (maskoff uses cv2.grabCut) and flip (https://github.com/bokeh/bokeh/issues/1666)
    img_cv = np.array(image.convert('RGB'))[::-1]
    img = img_cv[:, :, ::-1].copy()

    w, h = image.size
    size, crds = calculateGridCoords(w, h, init_pixels=8)
    if verbose: print('Box size', size)
    _, _, _, coords = maskoff(img, coords=crds, box_size=size, target_dim=image.size, pixel_score=0.05)
   
    p = figure(plot_width=800, plot_height=800, 
               x_range=(0,max(w,h)), y_range=(0,max(w,h)),
               tooltips=TOOLTIPS,
               title="Deteteced Gleason Patterns")
    
    im = image.convert("RGBA")
    # convert to numpy and flip (https://github.com/bokeh/bokeh/issues/1666)
    imarray = np.array(im)[::-1]
    p.image_rgba(image=[imarray], x=0,y=0, dw=w, dh=h)

    data = []
    # Classify the boxes
    for j, coord in enumerate(coords):
        x,y = coord
        patch = image.crop((x, y, x+size, y+size))
        c, l, preds = learn.predict(Image(T.ToTensor()(patch.convert('RGB'))))
        pname = learn.data.classes[l]
        tmp = dict(zip(learn.data.classes, ["{0:.2%}".format(p) for p in preds]))
        tmp['pred'] = pname
        tmp['x'] = x+size//2
        tmp['y'] = y+size//2
        tmp['line_clr'] = 'grey' if preds[l]<confidence else COLORMAP[pname]
        data.append(tmp)
        del patch
    end = time.time()
    df = pd.DataFrame(data)
    source = ColumnDataSource(data=df) 

    p.rect(x='x',y='y', width=size, height=size, line_color='line_clr', fill_color=None, source=source)
    
    if verbose: print('Took', end-start, 'to process image found', len(coords), 'patches')
    
    return p

Loading BokehJS ...

In [13]:


# defining widgets
karolinska_button = Button(description="Karolinska Biopsy", layout=Layout(height='auto', width='auto'))
radboud_button  = Button(description="Radboud Biopsy", layout=Layout(height='auto', width='auto'))
btn_upload = FileUpload(description="Upload Biopsy", multiple=False, layout=Layout(height='auto', width='auto'))
header_button = Button(description="Gleason Pattern Spotter", disabled=True, layout=Layout(height='auto', width='auto'))
footer_button = Button(description="Additional Information: Not Available", disabled=True, layout=Layout(height='auto', width='auto'))
output = Output(clear_output=True)

# defining event functions
def displayWaitMessage():
    footer_button.description = 'Please Wait A Few Seconds'
    footer_button.style.button_color = 'yellow'

def displayReadyness():
    footer_button.description = 'Ready For User Input'
    footer_button.style.button_color = 'lightgreen'

def processSampleImage(provider='karolinska'):
    output.clear_output()
    with output:
        displayWaitMessage()
        img = getSampleImageFromUrl(provider)
        p = bokehplotImageWithPatterns(img)
        show(p)
        displayReadyness()
    
def on_karolinska_button_clicked(b):
    processSampleImage(provider='karolinska')

def on_radboud_button_clicked(b):
    processSampleImage(provider='radboud')
        

def on_data_change(change):
    displayWaitMessage()
    img = PILImage.open(io.BytesIO(btn_upload.data[-1]))
    with output:
        p = bokehplotImageWithPatterns(img)
        show(p)
    btn_upload._counter = 0
    displayReadyness()

# adding events
karolinska_button.on_click(on_karolinska_button_clicked)
radboud_button.on_click(on_radboud_button_clicked)
btn_upload.observe(on_data_change, names=['data'])

# Layout and Style 
applayout = AppLayout(header=header_button,
                      left_sidebar=karolinska_button,
                      right_sidebar=radboud_button,
                      footer=VBox([btn_upload,footer_button], layout=Layout(height='auto', width='auto')))
karolinska_button.style.button_color = 'lightsalmon'
radboud_button.style.button_color = 'aliceblue'

display(VBox([applayout,
              output]))